In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

True

## Load Population Data

In [46]:
def get_posgres_connection():
    db_name = os.getenv("PSQL_DB_NAME")
    db_user = os.getenv("PSQL_DB_USER")
    db_password = os.getenv("PSQL_DB_PASSWORD")
    db_host = os.getenv("PSQL_DB_HOST")
    sql_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}')
    return sql_engine


In [2]:
sheets = ['hti_pop2019_adm0','hti_pop2019_adm1','hti_pop2019_adm2','hti_pop2019_adm3']

In [ ]:
for sheet in sheets:
    df = pd.read_excel("hti_adminboundaries_tabulardata.xlsx",sheet_name= sheet)
    psql_engine = get_posgres_connection()
    df.to_sql(name = sheet ,con=psql_engine,index=False,if_exists='replace')

## Load Shape Files

### Import the data to Ayiti Analytics PostGIS system
In order to load data to PostGIS, we need to identify the srid of the file prior to load.
####
1. Create a method to load the data 
    a. Convert data objects types for the geometry data.
    b. Load the data to PostGIS tables
2. Using the previously created function write a for-loop too load all the shape files. 
3. During the previous for-loop, we encounter an error. To resolve I will need to unpack the list and attempt to load each file with varying parameter values and changes to the database table.

### Get SRID
The SRID of the shape file is 4326

In [107]:
from osgeo import ogr
driver = ogr.GetDriverByName('ESRI Shapefile')
shape = driver.Open('hti_adm_cnigs_20181129/hti_admbnda_adm0_cnigs_20181129.shp')
layer= shape.GetLayer()
# the crs
crs = layer.GetSpatialRef()

# from Geometry
feature = layer.GetNextFeature()
geom = feature.GetGeometryRef()
print(crs)

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]


###### I look at the AUTHORITY  value-pair at the bottom

In [121]:
def get_srid(shpale_file):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    shape = driver.Open('hti_adm_cnigs_20181129/hti_admbnda_adm0_cnigs_20181129.shp')
    layer= shape.GetLayer()
    # the crs
    crs = layer.GetSpatialRef()

    # from Geometry
    feature = layer.GetNextFeature()
    geom = feature.GetGeometryRef()
    print(crs)

#### Load data to PostGIS

In [47]:
import geopandas as gpd
from geoalchemy2 import Geometry, WKTElement

In [13]:
gdf = gpd.read_file("hti_adm_cnigs_20181129/hti_admbnda_adm0_cnigs_20181129.shp")

In [14]:
gdf.head()

,Shape_Leng,Shape_Area,ADM0_EN,ADM0_FR,ADM0_HT,ADM0_PCODE,ADM0_REF,ADM0ALT1EN,ADM0ALT2EN,ADM0ALT1FR,ADM0ALT2FR,ADM0ALT1HT,ADM0ALT2HT,date,validOn,validTo,geometry
0,21.376764,2.310924,Haiti,Haïti,Ayiti,HT,None,None,None,None,None,None,None,2017-09-06,2018-11-29,None,"MULTIPOLYGON (((-73.70372 18.10930, -73.70315 ..."


In [15]:
gdf['geometry'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid="4326"))


/usr/local/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [50]:
gdf.head()

,Shape_Leng,Shape_Area,ADM0_EN,ADM0_FR,ADM0_HT,ADM0_PCODE,ADM0_REF,ADM0ALT1EN,ADM0ALT2EN,ADM0ALT1FR,ADM0ALT2FR,ADM0ALT1HT,ADM0ALT2HT,date,validOn,validTo,geom
0,21.376764,2.310924,Haiti,Haïti,Ayiti,HT,None,None,None,None,None,None,None,2017-09-06,2018-11-29,None,MULTIPOLYGON (((-73.70372158299995 18.10929985...


In [55]:
# Imports
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd

# Creating SQLAlchemy's engine to use
engine = get_posgres_connection()


#geodataframe = gpd.GeoDataFrame(pd.DataFrame.from_csv('<your dataframe source>'))
#... [do something with the geodataframe]
gdf = gpd.read_file("hti_adm_cnigs_20181129/hti_admbnda_adm0_cnigs_20181129.shp")

#geodataframe['geom'] = geodataframe['geometry'].apply(lambda x: WKTElement(x.wkt, srid=<your_SRID>)
gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid= 4326))


#drop the geometry column as it is now duplicative
#geodataframe.drop('geometry', 1, inplace=True)
gdf.drop('geometry', 1, inplace=True)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
gdf.to_sql("hti_shp_adm0", engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid= 4326)})

In [58]:
new_gdf = gpd.read_postgis("select * from hti_shp_adm0;",engine)
new_gdf

,Shape_Leng,Shape_Area,ADM0_EN,ADM0_FR,ADM0_HT,ADM0_PCODE,ADM0_REF,ADM0ALT1EN,ADM0ALT2EN,ADM0ALT1FR,ADM0ALT2FR,ADM0ALT1HT,ADM0ALT2HT,date,validOn,validTo,geom
0,21.376764,2.310924,Haiti,Haïti,Ayiti,HT,None,None,None,None,None,None,None,2017-09-06,2018-11-29,None,"MULTIPOLYGON (((-73.70372 18.10930, -73.70315 ..."


In [99]:
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd

def load_shape_file_to_db(shape_file,table,engine):
    
    # Load shape file with geopandas
    gdf = gpd.read_file(shape_file)

    #Convert geometry column to WKTElement
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid= 4326))


    #drop the geometry column as it is now duplicative
    gdf.drop('geometry', 1, inplace=True)

    # Use 'dtype' to specify column's type
    # For the geom column, we will use GeoAlchemy's type 'Geometry'
    gdf.to_sql(table, engine, if_exists='replace', index=False, 
                             dtype={'geom': Geometry('MULTIPOLYGON', srid= 4326)})
    return None

In [60]:
import glob

In [64]:
glob.glob("hti_adm_cnigs_20181129/*.shp")

['hti_adm_cnigs_20181129/hti_admbnda_adm2_cnigs_20181129.shp',
 'hti_adm_cnigs_20181129/hti_admbnda_adm1_cnigs_20181129.shp',
 'hti_adm_cnigs_20181129/hti_admbnda_adm0_cnigs_20181129.shp',
 'hti_adm_cnigs_20181129/hti_admbndl_admALL_cnigs_itos_20181129.shp',
 'hti_adm_cnigs_20181129/hti_admbndp_admALL_cnigs_itos_20181129.shp',
 'hti_adm_cnigs_20181129/hti_admbnda_adm3_cnigs_20181129.shp']

In [65]:
shape_files = [
    ('hti_adm_cnigs_20181129/hti_admbnda_adm0_cnigs_20181129.shp','hti_shp_adm0'),
    ('hti_adm_cnigs_20181129/hti_admbnda_adm1_cnigs_20181129.shp','hti_shp_adm1'),
    ('hti_adm_cnigs_20181129/hti_admbnda_adm2_cnigs_20181129.shp','hti_shp_adm2'),
    ('hti_adm_cnigs_20181129/hti_admbnda_adm3_cnigs_20181129.shp','hti_shp_adm3'),
]

In [77]:
# I want to load each shape file identified in my list with the tuple-pair values
for file in shape_files:
    file_path =  file[0]
    file_name = file[1]
    psql_engine = get_posgres_connection()
    load_shape_file_to_db(file_path,file_name,psql_engine)

DataError: (psycopg2.errors.InvalidParameterValue) Geometry type (MultiPolygon) does not match column type (Polygon)

[SQL: INSERT INTO hti_shp_adm0 ("Shape_Leng", "Shape_Area", "ADM0_EN", "ADM0_FR", "ADM0_HT", "ADM0_PCODE", "ADM0_REF", "ADM0ALT1EN", "ADM0ALT2EN", "ADM0ALT1FR", "ADM0ALT2FR", "ADM0ALT1HT", "ADM0ALT2HT", date, "validOn", "validTo", geom) VALUES (%(Shape_Leng)s, %(Shape_Area)s, %(ADM0_EN)s, %(ADM0_FR)s, %(ADM0_HT)s, %(ADM0_PCODE)s, %(ADM0_REF)s, %(ADM0ALT1EN)s, %(ADM0ALT2EN)s, %(ADM0ALT1FR)s, %(ADM0ALT2FR)s, %(ADM0ALT1HT)s, %(ADM0ALT2HT)s, %(date)s, %(validOn)s, %(validTo)s, ST_GeomFromEWKT(%(geom)s))]
[parameters: {'Shape_Leng': 21.3767639454, 'Shape_Area': 2.31092395464, 'ADM0_EN': 'Haiti', 'ADM0_FR': 'Haïti', 'ADM0_HT': 'Ayiti', 'ADM0_PCODE': 'HT', 'ADM0_REF': None, 'ADM0ALT1EN': None, 'ADM0ALT2EN': None, 'ADM0ALT1FR': None, 'ADM0ALT2FR': None, 'ADM0ALT1HT': None, 'ADM0ALT2HT': None, 'date': '2017-09-06', 'validOn': '2018-11-29', 'validTo': None, 'geom': 'SRID=4326;MUL

# Fixing importation of files adm1 and adm2
Below I fix the import error by loading each file individually and attempting to load it with different parameters.In order to load the data for adm1 and adm2 boundaries which have both polygon and multipolygons in the geometry column. To load the datasets I had to allow the column the PostGIS table to have both polygon and multipolygons in a single column by running the following commands after I ran the functions the first time with `if_exists = 'replace`.

 `ALTER TABLE hti_shp_adm1 ALTER COLUMN geom TYPE geometry(Geometry,4326);
ALTER TABLE hti_shp_adm2 ALTER COLUMN geom TYPE geometry(Geometry,4326);`
I ran these sql statements after I received an error, I then proceed to rerun the functions but change `if_exists ='append'` since the tables had already been created.

I can improve this code by simply running the sql to generate the tables first, run the code to alter the columns in the respective tables and then load the data.


In [ ]:
def load_shape_file_to_db(shape_file,table,engine,geom_type ='MULTIPOLYGON',if_exists ='replace',srid = 4326):
    """
    This function will load a shape file to a PostGIS database system.
    shape_file str: path to shapefile
    table str:name of database file to load to POSTGIS
    engine sqlalchemy.Engine: A sqlAlchemy POSTGIS Engine Object
    if_exists str: method of upload to database, default is "replace"
    srid int: format ID of shape files, this can be discovered by using the
    
    """
    # Load shape file with geopandas
    gdf = gpd.read_file(shape_file)

    #Convert geometry column to WKTElement
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid= 4326))


    #drop the geometry column as it is now duplicative
    gdf.drop('geometry', 1, inplace=True)

    # Use 'dtype' to specify column's type
    # For the geom column, we will use GeoAlchemy's type 'Geometry'
    gdf.to_sql(table, engine, if_exists= if_exists, index=False, 
                             dtype={'geom': Geometry(geom_type, srid= 4326)})
    return None

In [90]:
gdf = gpd.read_file(shape_files[1][0])
gdf.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_FR,ADM1_HT,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM1ALT1FR,...,ADM1ALT1HT,ADM1ALT2HT,ADM0_EN,ADM0_FR,ADM0_HT,ADM0_PCODE,date,validOn,validTo,geometry
0,4.993649,0.417618,Artibonite,Artibonite,Latibonit,HT05,None,None,None,None,...,None,None,Haiti,Haïti,Ayiti,HT,2017-09-06,2018-11-29,None,"POLYGON ((-72.69573 19.81251, -72.69408 19.811..."
1,3.601817,0.296991,Centre,Centre,Sant,HT06,None,None,None,None,...,None,None,Haiti,Haïti,Ayiti,HT,2017-09-06,2018-11-29,None,"POLYGON ((-71.98404 19.33367, -71.98348 19.332..."
2,2.796977,0.163747,Grande'Anse,Grande'Anse,Grandans,HT08,GrandeAnse,None,None,None,...,None,None,Haiti,Haïti,Ayiti,HT,2017-09-06,2018-11-29,None,"MULTIPOLYGON (((-73.75492 18.64331, -73.75429 ..."
3,3.294820,0.104843,Nippes,Nippes,Nip,HT10,None,None,None,None,...,None,None,Haiti,Haïti,Ayiti,HT,2017-09-06,2018-11-29,None,"POLYGON ((-73.56737 18.58722, -73.56536 18.585..."
4,3.834603,0.181067,North,Nord,Nò,HT03,None,None,None,None,...,None,None,Haiti,Haïti,Ayiti,HT,2017-09-06,2018-11-29,None,"POLYGON ((-72.55529 19.87769, -72.55468 19.877..."


In [96]:
# Country
file_id = 0
psql_engine = get_posgres_connection()
load_shape_file_to_db(shape_files[file_id][0],shape_files[file_id][1],psql_engine)

In [103]:
# Region
file_id = 1
psql_engine = get_posgres_connection()
load_shape_file_to_db(shape_files[file_id][0],
                      shape_files[file_id][1],
                      psql_engine,
                      geom_type="POLYGON",
                      if_exists ='append')





In [105]:
# Department
file_id = 2
psql_engine = get_posgres_connection()
load_shape_file_to_db(shape_files[file_id][0],shape_files[file_id][1],psql_engine,geom_type="POLYGON",if_exists ='append')

In [93]:
# Commune
file_id = 3
psql_engine = get_posgres_connection()
load_shape_file_to_db(shape_files[file_id][0],shape_files[file_id][1],psql_engine,geom_type="POLYGON")